In [1]:
import datetime
import os
import pandas as pd
import numpy as np
from pprint import pprint
import requests
# Scrappings tools for downloading
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import re

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
qualifiying_results = pd.read_csv('../../scrapping/data/qualifiying_results.csv')

In [4]:
# Filter the dataset from 2011 to 2023 because it is the current range ow our weather DF
df = qualifiying_results[qualifiying_results['year'].between(2011, 2023)]

In [5]:
# Convert Laps from double to integer for removing the flotting 0 at the end of each lap
df['laps'] = df['laps'].astype(int)

In [6]:
df.head()

,position,number,driver,team,time,Q1,Q2,Q3,laps,year,round
11860,1,1,Sebastian Vettel,RED BULL RACING RENAULT,NaN,1:25.296,1:24.090,1:23.529,16,2011,1
11861,2,3,Lewis Hamilton,MCLAREN MERCEDES,NaN,1:25.384,1:24.595,1:24.307,19,2011,1
11862,3,2,Mark Webber,RED BULL RACING RENAULT,NaN,1:25.900,1:24.658,1:24.395,15,2011,1
11863,4,4,Jenson Button,MCLAREN MERCEDES,NaN,1:25.886,1:24.957,1:24.779,18,2011,1
11864,5,5,Fernando Alonso,FERRARI,NaN,1:25.707,1:25.242,1:24.974,19,2011,1


In [61]:
df['have_pole_position'] = df['position'].apply(lambda x : True if x == '1' else False)
df['is_in_qualifiying_top5'] = df['position'].apply(lambda x : True if x in ['1', '2', '3', '4', '5'] else False)
df['is_in_Q3'] = df['Q3'].apply(lambda x : True if x is not np.nan else False)
df['driver'] = df['driver'].apply(lambda x: x.replace("Sergio Perez", "Sergio Pérez"))

In [69]:
"Sergio Pérez".upper()

'SERGIO PÉREZ'

In [62]:
df.head()

,position,number,driver,team,time,Q1,Q2,Q3,laps,year,round,have_pole_position,is_in_qualifiying_top5,is_in_Q3
11860,1,1,Sebastian Vettel,RED BULL RACING RENAULT,NaN,1:25.296,1:24.090,1:23.529,16,2011,1,True,True,True
11861,2,3,Lewis Hamilton,MCLAREN MERCEDES,NaN,1:25.384,1:24.595,1:24.307,19,2011,1,False,True,True
11862,3,2,Mark Webber,RED BULL RACING RENAULT,NaN,1:25.900,1:24.658,1:24.395,15,2011,1,False,True,True
11863,4,4,Jenson Button,MCLAREN MERCEDES,NaN,1:25.886,1:24.957,1:24.779,18,2011,1,False,True,True
11864,5,5,Fernando Alonso,FERRARI,NaN,1:25.707,1:25.242,1:24.974,19,2011,1,False,True,True


In [63]:
# driver_indicators = df[['position', 'number', 'driver', 'team', 'laps', 'year', 'round', 'Q3', 'have_pole_position', 'is_in_qualifiying_top5', 'is_in_Q3']]
driver_indicators = df[['position', 'driver', 'team', 'laps', 'year', 'round', 'Q3', 'have_pole_position', 'is_in_qualifiying_top5', 'is_in_Q3']]
driver_indicators= driver_indicators.rename(columns={'Q3': 'qualifiying_time'})

In [ ]:
driver_indicators[driver_indicators['year'] == 2023][driver_indicators['round'] == 13]

In [71]:
driver_indicators.tail(40)

,position,driver,team,laps,year,round,qualifiying_time,have_pole_position,is_in_qualifiying_top5,is_in_Q3
17097,1,Lewis Hamilton,MERCEDES,20,2023,12,1:16.609,True,True,True
17098,2,Max Verstappen,RED BULL RACING HONDA RBPT,21,2023,12,1:16.612,False,True,True
17099,3,Lando Norris,MCLAREN MERCEDES,20,2023,12,1:16.694,False,True,True
17100,4,Oscar Piastri,MCLAREN MERCEDES,20,2023,12,1:16.905,False,True,True
17101,5,Zhou Guanyu,ALFA ROMEO FERRARI,22,2023,12,1:16.971,False,True,True
17102,6,Charles Leclerc,FERRARI,21,2023,12,1:16.992,False,False,True
17103,7,Valtteri Bottas,ALFA ROMEO FERRARI,20,2023,12,1:17.034,False,False,True
17104,8,Fernando Alonso,ASTON MARTIN ARAMCO MERCEDES,21,2023,12,1:17.035,False,False,True
17105,9,Sergio Pérez,RED BULL RACING HONDA RBPT,23,2023,12,1:17.045,False,False,True
17106,10,Nico Hulkenberg,HAAS FERRARI,20,2023,12,1:17.186,False,False,True


In [35]:
races_results = pd.read_csv('../../scrapping/data/races_results.csv')

In [39]:
races_results = races_results[['season', 'round', 'circuit_id', 'driver', 'date_of_birth',
       'nationality', 'constructor', 'driver_number', 'time', 'status',
       'points', 'podium']]

In [47]:
races_results= races_results.rename(columns={'season': 'year'})

In [44]:
races_results = races_results[races_results['year'].between(2011, 2023)]

In [72]:
races_results.tail(40)

,year,round,circuit_id,driver,date_of_birth,nationality,constructor,driver_number,time,status,points,podium
25587,2023,11,hungaroring,Max Verstappen,1997-09-30,Dutch,red_bull,33.0,5888634.0,Finished,26.0,1
25588,2023,11,hungaroring,Lando Norris,1999-11-13,British,mclaren,4.0,5922365.0,Finished,18.0,2
25589,2023,11,hungaroring,Sergio Pérez,1990-01-26,Mexican,red_bull,11.0,5926237.0,Finished,15.0,3
25590,2023,11,hungaroring,Lewis Hamilton,1985-01-07,British,mercedes,44.0,5927768.0,Finished,12.0,4
25591,2023,11,hungaroring,Oscar Piastri,2001-04-06,Australian,mclaren,81.0,5951206.0,Finished,10.0,5
25592,2023,11,hungaroring,George Russell,1998-02-15,British,mercedes,63.0,5954459.0,Finished,8.0,6
25593,2023,11,hungaroring,Charles Leclerc,1997-10-16,Monegasque,ferrari,16.0,5958951.0,Finished,6.0,7
25594,2023,11,hungaroring,Carlos Sainz,1994-09-01,Spanish,ferrari,55.0,5959707.0,Finished,4.0,8
25595,2023,11,hungaroring,Fernando Alonso,1981-07-29,Spanish,aston_martin,14.0,5964343.0,Finished,2.0,9
25596,2023,11,hungaroring,Lance Stroll,1998-10-29,Canadian,aston_martin,18.0,NaN,+1 Lap,1.0,10


In [65]:
df1 = pd.merge(races_results, driver_indicators, how='inner', on=['year', 'round', 'driver'])

In [67]:
df1[df1['driver'] == 'Sergio Pérez']

,year,round,circuit_id,driver,date_of_birth,nationality,constructor,driver_number,time,status,points,podium,position,team,laps,qualifiying_time,have_pole_position,is_in_qualifiying_top5,is_in_Q3
19,2011,1,albert_park,Sergio Pérez,1990-01-26,Mexican,sauber,11.0,NaN,Disqualified,0.0,22,13,SAUBER FERRARI,9,NaN,False,False,False
40,2011,2,sepang,Sergio Pérez,1990-01-26,Mexican,sauber,11.0,NaN,Accident,0.0,21,16,SAUBER FERRARI,10,NaN,False,False,False
59,2011,3,shanghai,Sergio Pérez,1990-01-26,Mexican,sauber,11.0,NaN,+1 Lap,0.0,17,12,SAUBER FERRARI,16,NaN,False,False,False
78,2011,4,istanbul,Sergio Pérez,1990-01-26,Mexican,sauber,11.0,NaN,+1 Lap,0.0,14,15,SAUBER FERRARI,11,NaN,False,False,False
95,2011,5,catalunya,Sergio Pérez,1990-01-26,Mexican,sauber,11.0,NaN,+1 Lap,2.0,9,12,SAUBER FERRARI,9,NaN,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4800,2023,8,villeneuve,Sergio Pérez,1990-01-26,Mexican,red_bull,11.0,5689376.0,Finished,9.0,6,11,RED BULL RACING HONDA RBPT,17,NaN,False,False,False
4814,2023,9,red_bull_ring,Sergio Pérez,1990-01-26,Mexican,red_bull,11.0,5150795.0,Finished,15.0,3,12,RED BULL RACING HONDA RBPT,23,NaN,False,False,False
4834,2023,10,silverstone,Sergio Pérez,1990-01-26,Mexican,red_bull,11.0,5129820.0,Finished,8.0,6,15,RED BULL RACING HONDA RBPT,19,NaN,False,False,False
4848,2023,11,hungaroring,Sergio Pérez,1990-01-26,Mexican,red_bull,11.0,5926237.0,Finished,15.0,3,15,RED BULL RACING HONDA RBPT,11,NaN,False,False,False


In [68]:
df1.tail(20)

,year,round,circuit_id,driver,date_of_birth,nationality,constructor,driver_number,time,status,points,podium,position,team,laps,qualifiying_time,have_pole_position,is_in_qualifiying_top5,is_in_Q3
4861,2023,11,hungaroring,Esteban Ocon,1996-09-17,French,alpine,31.0,NaN,Collision damage,0.0,19,13,ALPINE RENAULT,18,NaN,False,False,False
4862,2023,11,hungaroring,Pierre Gasly,1996-02-07,French,alpine,10.0,NaN,Collision damage,0.0,20,10,ALPINE RENAULT,24,1:27.689,False,False,True
4863,2023,12,spa,Max Verstappen,1997-09-30,Dutch,red_bull,33.0,4950450.0,Finished,25.0,1,2,RED BULL RACING HONDA RBPT,21,1:16.612,False,True,True
4864,2023,12,spa,Sergio Pérez,1990-01-26,Mexican,red_bull,11.0,4972755.0,Finished,18.0,2,9,RED BULL RACING HONDA RBPT,23,1:17.045,False,False,True
4865,2023,12,spa,Charles Leclerc,1997-10-16,Monegasque,ferrari,16.0,4982709.0,Finished,15.0,3,6,FERRARI,21,1:16.992,False,False,True
4866,2023,12,spa,Lewis Hamilton,1985-01-07,British,mercedes,44.0,5000121.0,Finished,13.0,4,1,MERCEDES,20,1:16.609,True,True,True
4867,2023,12,spa,Fernando Alonso,1981-07-29,Spanish,aston_martin,14.0,5006634.0,Finished,10.0,5,8,ASTON MARTIN ARAMCO MERCEDES,21,1:17.035,False,False,True
4868,2023,12,spa,George Russell,1998-02-15,British,mercedes,63.0,5013551.0,Finished,8.0,6,18,MERCEDES,9,NaN,False,False,False
4869,2023,12,spa,Lando Norris,1999-11-13,British,mclaren,4.0,5024169.0,Finished,6.0,7,3,MCLAREN MERCEDES,20,1:16.694,False,True,True
4870,2023,12,spa,Esteban Ocon,1996-09-17,French,alpine,31.0,5025169.0,Finished,4.0,8,12,ALPINE RENAULT,16,NaN,False,False,False
